# Submodule 2: Preprocessing and Quality Control

<img src="images/LessonPlan2.jpg" alt="Drawing" width=1000 />

## Overview & Purpose
## Filtering, Signal Visualization, and Peak Identification
This submodule introduces concepts for post map filtering, visualization of signal, identifying binding sites by calling peaks, detection of sequence motifs, and differential peak analysis.  

To demonstrate the process, we will build on the analysis performed  in submodule 1, where we preprocesses and mapped ChIP-seq, CUT&RUN, or CUT&Tag data comparing BAF inhibitor or mutant to a control sample. 

As a reminder, this module covers the processing of the data from three distinct but similar methods using downsampled data to improve runtime speed. The original data was published in :

Weber CM, et al. mSWI/SNF promotes Polycomb repression both directly and through genome-wide redistribution. Nat Struct Mol Biol. 2021  PMID: [34117481](https://pubmed.ncbi.nlm.nih.gov/34117481/)

Brahma S, Henikoff S. The BAF chromatin remodeler synergizes with RNA polymerase II and transcription factors to evict nucleosomes. Nat Genet. 2024 PMID: [38049663](https://pubmed.ncbi.nlm.nih.gov/38049663/)

Note that to allow faster processing we have limited the reads to that of a single chromosome (chr4).  

### Ways to use this module
If you used submodule 1, you may recall how to navigate through the module. Throughout this module, we have color-coded commands according to ChIP-seq, CUT&RUN, and CUT&Tag. Therefore this module can be used to learn about the processing of each method individually, to compare each method to the others, or you can follow the colored commands to only process one type, either ChIP-seq, CUT&RUN, or CUT&Tag.
Commands for each method will be designated by an individual logo before the command, just like the following examples

<img src="images/ChIPseqLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [ ]:
#run this cell for ChIP-seq
print("Code for ChIP-seq will be placed after the above image. Run these cells if performing ChIP-seq analysis.")

<img src="images/CUT&RUNLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [ ]:
#run this cell for CUT&RUN
print("Code for CUT&RUN will be placed after the above image. Run these cells if performing CUT&RUN analysis.")

<img src="images/CUT&TagLogo.jpg" alt="Drawing" style="width: 250px;" align="left"/>

In [ ]:
#run this cell for CUT&Tag
print("Code for CUT&Tag will be placed after the above image. Run these cells if performing CUT&Tag analysis.")

<div class="alert alert-block alert-success" style="font-size:100%">
<span style="color:black"> By following the colors/images, you can run one, two, or all three types of analyses.</span>
</div>

### Required Files
In this stage of the module, you will use the sam files that are the output from submodule 1 (We also provide them if you skipped submodule 1). You can also use this module on your own data or any published ChIP-seq, CUT&RUN, or CUT&Tag dataset. 

<div class="alert-info" style="font-size:200%">
STEP 1: Set Up Environment
</div>

Initial items to configure your Cloud environment. In this step we will use conda to install the following packages:

Quality filtering and deduplication:
[samtools](https://anaconda.org/bioconda/samtools), [picard](https://anaconda.org/bioconda/picard)

Visualization:
[igv-notebook](https://igv.org/), [deeptools](https://deeptools.readthedocs.io/en/develop/)

Peak identification:
[macs3](https://github.com/macs3-project/MACS)

Motif analysis:
[Homer](http://homer.ucsd.edu/homer/motif/)

Differential peak calling:
[MAnorm](https://github.com/shao-lab/MAnorm)

In [ ]:
#First let's install mamba to configure our environment
! curl -L -O "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"
! bash Miniforge3-$(uname)-$(uname -m).sh -u -b -p $HOME/mambaforge
print("done")

In [ ]:
#now let's install several required packages
!mamba install -c bioconda picard samtools deeptools bedtools macs3 meme manorm -y
!pip install jupyterquiz==2.0.7 jupytercards
!pip install --user igv-notebook
print("done")

<div class="alert alert-block alert-warning" style="font-size:100%">
<span style="color:black"> The next cell will restart the kernel, so you will see the following message once you run the cell. When this image appears, click okay, and then keep going onto the next cell.</span>
</div>

<img src="./images/restartkernel.jpg" alt="Drawing" style="width: 500px;" align="left"/>

In [ ]:
#Restarts the kernel to implement changes 
import os
os._exit(00)

<div class="alert alert-block alert-warning" style="font-size:100%">
<span style="color:black"> After the kernel restarts continue from here.</span>
</div>

In [ ]:
#Now let's import packages that we installed
numthreads=!lscpu | grep '^CPU(s)'| awk '{print $2-1}'
numthreadsint = int(numthreads[0])
import sys
import os
from jupyterquiz import display_quiz
from IPython.display import IFrame
#from IPython.display import display
from IPython.display import Image
from jupytercards import display_flashcards
import igv_notebook
import pandas as pd
#import modules for matching-type quiz
%cd questions
from quiz_module import run_quiz
%cd ../
import json
import ipywidgets as widgets
from IPython.display import display
import random
print("done")

In [ ]:
wd="~/SageMaker/SandboxChromatinOccupancy"
%cd $wd
#show which folder you are working in. 
!pwd

In [ ]:
# These commands move into our Tutorial 1 directory and create our subdirectory structure.
!mkdir -p $wd/Submodule2/
%cd $wd/Submodule2/
!mkdir -p $wd/Submodule2/Filtering
!mkdir -p $wd/Submodule2/Visualization
!mkdir -p $wd/Submodule2/Peaks
!mkdir -p $wd/Submodule2/Motifs

In [ ]:
#Let's copy and extract our tutorial files
!wget https://chromatinoccupancytutorial.s3.us-east-2.amazonaws.com/Submodule2.zip
!unzip Submodule2.zip
print("done")

In [ ]:
#you should see four ChIP-seq sam files.
!ls $wd/Submodule2/ChIPSeqSam

<div class="alert-info" style="font-size:200%">
STEP 2: Filter and convert to bam
</div>

In submodule 1, we introduced the sam format. 

<img src="images/samformat.jpg" alt="Drawing" style="width: 800px;"/>

We'll start by filtering out low quality mappings, sort the file by chromosomal coordinate, and output the result as a bam file.

In [ ]:
# This will convert to bam by using samtools view with the -b option. The h and S option tells samtools that the file has a header and is in sam format. We will pipe this to samtools sort. Pay attention to the "-" at the end of the sort command which tells samtools to use stdin.
!samtools view -q 10 -bhS $wd/Submodule2/ChIPSeqSam/H3K27ac_ChIPseq_noaux.sam | samtools sort -o $wd/Submodule2/Filtering/H3K27ac_ChIPseq_noaux.bam - 
print("done")
#We'll do the same command for our bafi sample.
!samtools view -q 10 -bhS $wd/Submodule2/ChIPSeqSam/H3K27ac_ChIPseq_aux.sam | samtools sort -o $wd/Submodule2/Filtering/H3K27ac_ChIPseq_aux.bam - 
print("done")

You may have noticed the parameters -bhS and -q 10 in the above commands. Briefly, -bhS describes aspects of the file to samtools, such that you want to output a bam file (the b option), that it has a header (the h option), and that it is currently in sam format (the S option). We also specified -q 10 which removes reads with a mapping score <= 10. 

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question: Click on the correct answer in the following cell.
</div>

In [ ]:
%cd $wd/Submodule2
display_quiz("../questions/mappingquality.json")

<div class="alert-info" style="font-size:200%">
STEP 3: Removal of Duplicates
</div>
It's important to remove duplicates from our reads because part of the ChIP-seq method includes a PCR step for library amplification. This can create biases in the data resulting from PCR duplicates. To understand how PCR duplicates can affect the analysis, let's jump ahead a bit. Occupancy is represented by "peaks" of signal.

<img src="images/peaksofsignal.jpg" alt="Drawing" style="width: 500px;"/>

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question: Click on the correct answer in the following cell.
</div>

In [ ]:
%cd $wd/Submodule2
display_quiz("../questions/duplicateQuiz.json")

Okay, let's remove these duplicates using picard.

In [ ]:
%cd $wd/Submodule2
# This will take the sorted bam file and remove duplicates, saving a new bam file and a summary in a text file.
!picard MarkDuplicates REMOVE_DUPLICATES=TRUE I=Filtering/H3K27ac_ChIPseq_noaux.bam O=Filtering/H3K27ac_ChIPseq_noauxdedup.bam METRICS_FILE=Filtering/H3K27ac_ChIPseq_ctldedup_metrics.txt 2> Filtering/PicardLog.txt
print("done")
#Let's do it again, this time for bafi.
!picard MarkDuplicates REMOVE_DUPLICATES=TRUE I=Filtering/H3K27ac_ChIPseq_aux.bam O=Filtering/H3K27ac_ChIPseq_auxdedup.bam METRICS_FILE=Filtering/H3K27ac_ChIPseq_auxdedup_metrics.txt 2> Filtering/PicardLog.txt
print("done")

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question: Match the tool that we used for each step in the analysis.
</div>

In [ ]:
#Run for the quiz
%cd $wd/Submodule2
run_quiz("../questions/postprocessingtools.json", instant_feedback=True, shuffle_questions=False, shuffle_answers=True)

<div class="alert-info" style="font-size:200%">
STEP 4: Visualization
</div>

Let's view the first line of one of our bam files:

In [ ]:
!samtools view $wd/Submodule2/Filtering/H3K27ac_ChIPseq_noauxdedup.bam | head -n 3 
# Note that there will be an error message because we are breaking a pipe by printing only the first 3 lines. Please ignore the error message.

### While we can see the coordinates of each read, we will need a better way of visualizing the results. In this step we will create a binary file that summarizes the pileup of reads at each base-pair along our genome, in [bigwig](http://genome.ucsc.edu/goldenPath/help/bigWig.html) format. 

To create the bigwig files let's use the command bamCoverage, part of the [deeptools](https://deeptools.readthedocs.io/en/develop/) package.

<img src="images/balance.jpg" alt="Drawing" style="width: 100px;"/>

But first, let's talk about normalization.  What would these look like if we sequenced 100 million reads for one sample but only 1 million reads for another? 

We need to normalize our signal by the depth of sequencing. We can do so by dividing the count of reads at each coordinate by the total (per million). We can do this at the same time as creating the bigwig.

This may take a few minutes: 🕘

In [ ]:
# First we need to create an index of our bam file.
!samtools index $wd/Submodule2/Filtering/H3K27ac_ChIPseq_noauxdedup.bam

# Then we can create a bigwig file of the control sample.
!bamCoverage -b $wd/Submodule2/Filtering/H3K27ac_ChIPseq_noauxdedup.bam -o $wd/Submodule2/Visualization/H3K27ac_ChIPseq_noauxdedup.bw -bs 50 --normalizeUsing BPM 2> $wd/Submodule2/Visualization/bamCovLog_noaux.txt

# Now let's rerun the commands for our other sample.
!samtools index $wd/Submodule2/Filtering/H3K27ac_ChIPseq_auxdedup.bam
!bamCoverage -b $wd/Submodule2/Filtering/H3K27ac_ChIPseq_auxdedup.bam -o $wd/Submodule2/Visualization/H3K27ac_ChIPseq_auxdedup.bw -bs 50 --normalizeUsing BPM 2> $wd/Submodule2/Visualization/bamCovLog_aux.txt

print("done")

In the above example we specify the bam file name after -b and the output file name after -o. 

We specified -bs 50, which tells bamCoverage to summarize the reads at 50 bp resolution.

You could also specify the number of threads to use with -p.

Lastly, we specified --normalizeUsing BPM. BPM stands for Bins Per Million mapped reads. What do you think this normalization does?

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question: Click on the correct answer in following cell.
</div>

In [ ]:
%cd $wd/Submodule2
display_quiz("../questions/BPMnorm.json")

<div class="alert-info" style="font-size:150%">
Genome Browser
</div>

Now that we have our bigwig files, we can visualize the signal in a genome browser. We'll use [igv](https://igv.org/) in this example.

In [ ]:
%cd $wd/
igv_notebook.init()
myigv = igv_notebook.Browser(
    {
        "genome": "mm39",
        "locus": "chr4:44,000,000-47,000,000"
    }
)
myigv.load_track(
{
        "name": "H3K27ac_ChIPseq_noaux",
        "url": "Submodule2/Visualization/H3K27ac_ChIPseq_noauxdedup.bw",
        "format": "bigwig",
        "type": "wig"
    }
    
)
myigv.load_track(
{
        "name": "H3K27ac_ChIPseq_aux",
        "url": "Submodule2/Visualization/H3K27ac_ChIPseq_auxdedup.bw",
        "format": "bigwig",
        "type": "wig"
    }
    
)

This will load in the signal into IGV and allow you to browse the genome. Feel free to play around with this. More instructions can be found on the [IGV](https://igv.org/) website.

Click the "gear" icon on the right of each track to customize the color, name, height, etc... 

The default is to autoscale the tracks. But this means that our samples have different y-axis scales! Because we BPM normalized, we should set the data range to be the same for each sample.

<img src="images/igvdatarange.jpg" alt="Drawing" style="width: 1000px;"/>

<div class="alert-info" style="font-size:120%">
Average Profiles
</div>

In addition to browsing, we can make average profiles of signal across specific regions. For example, let's check if there is any signal enrichment around genes. Let's test this using [deeptools](https://anaconda.org/bioconda/deeptools).

Deeptools takes in a bigwig file representing the signal. It also takes a bed file representing the features across which one wants to average the signal. In our case the bed file will be composed of gene annotations. Creating the profile will occur in two steps. The first is to create the summarized matrix, while the second plots that data.

In the following example, we'll use options which scale genes to a similar size and plots the average signal across the region.

This may take a few minutes: 🕘

In [ ]:
%cd $wd
# -S option specifies the bigwig signal file, where we can specify multiple separated by spaces. -R option specifies the genome annotation bed file. -a and -b specify how many bp to plot on either side. 
!computeMatrix scale-regions -S Submodule2/Visualization/H3K27ac_ChIPseq_noauxdedup.bw Submodule2/Visualization/H3K27ac_ChIPseq_auxdedup.bw -R Submodule2/GenomeAnnotations/mm39v36_chr4_genes.bed -o Submodule2/Visualization/GeneprofileMatrix -a 2000 -b 2000 --regionBodyLength 2000
print("done creating the matrix... plotting....")
!plotProfile -m Submodule2/Visualization/GeneprofileMatrix -o Submodule2/Visualization/Geneprofile.png --perGroup
print("done")

Let's view the output:

In [ ]:
%cd $wd
Image(url= "Submodule2/Visualization/Geneprofile.png", width=400, height=400)

Later, we'll do some downstream analysis, which will identify differential occupancy and determine where they are relative to genomic features.

<div class="alert-info" style="font-size:200%">
STEP 3: Peak Detection
</div>

<img src="images/mountain.jpg" alt="Drawing" style="width: 100px;"/>

Occupied sites are loci with a pileup of reads in "Peaks". In the next steps, we'll identify these sites genome-wide.

<img src="images/peaksofsignal.jpg" alt="Drawing" style="width: 400px;"/>

To call peaks, we'll use macs3.

<div class="alert-warning" style="font-size:200%" color="black">
A note about controls
</div>
In ChIP-seq, CUT&RUN, and CUT&Tag it is important to control for non-specific signal enrichment. The processing steps up until this point are exactly the same for those control samples, but we'll use the control samples during peak calling to provide a background level. We've prepared the bam files for the Input control samples for you, and specify them in the following command.  
</div>

In [ ]:
!macs3 callpeak -t $wd/Submodule2/Filtering/H3K27ac_ChIPseq_noauxdedup.bam -c $wd/Submodule2/ChIPSeqInputBam/Input_ChIPseq_noaux_dedup.bam -f BAM -g mm -n $wd/Submodule2/Peaks/H3K27ac_ChIPseq_noaux --nomodel
#Do it again for the other sample.
!macs3 callpeak -t $wd/Submodule2/Filtering/H3K27ac_ChIPseq_auxdedup.bam -c $wd/Submodule2/ChIPSeqInputBam/Input_ChIPseq_aux_dedup.bam -f BAM -g mm -n $wd/Submodule2/Peaks/H3K27ac_ChIPseq_aux --nomodel

#### Let's count how many "peaks" it called. 

In [ ]:
!wc -l $wd/Submodule2/Peaks/H3K27ac_ChIPseq_noaux_peaks.narrowPeak
!wc -l $wd/Submodule2/Peaks/H3K27ac_ChIPseq_aux_peaks.narrowPeak

From the counts above, it looks like auxin-mediated degradation of BAF results in fewer identified H3K27ac sites. Later, we'll use better statistical analysis to identify differences quantitatively

#### Let's also visualize the peak calls.

In [ ]:
%cd $wd
igv_notebook.init()
myigv = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr4:39,000,000-42,000,000"
    }
)
myigv.load_track(
{
        "name": "H3K27ac_ChIPseq_noaux",
        "url": "Submodule2/Visualization/H3K27ac_ChIPseq_noauxdedup.bw",
        "format": "bigwig",
        "type": "wig"
    }
    
)
myigv.load_track(
{
        "name": "H3K27ac -auxin peaks",
        "url": "Submodule2/Peaks/H3K27ac_ChIPseq_noaux_peaks.narrowPeak",
        "format": "bed",
        "type": "annotation"
    }
    
)
myigv.load_track(
{
        "name": "H3K27ac_ChIPseq_aux",
        "url": "Submodule2/Visualization/H3K27ac_ChIPseq_auxdedup.bw",
        "format": "bigwig",
        "type": "wig"
    }
    
)
myigv.load_track(
{
        "name": "H3K27ac +auxin peaks",
        "url": "Submodule2/Peaks/H3K27ac_ChIPseq_aux_peaks.narrowPeak",
        "format": "bed",
        "type": "annotation"
    }
    
)

<div class="alert-info" style="font-size:200%">
STEP 4: Differential Peaks
</div>

We want to identify differences in occupancy. Many people new to analysis will simply intersect peak calls, but we strongly recommend to not take that approach! 

Peak identification is not perfect, and identification of a peak in one sample, does not mean there wasn't enriched signal in the other! Instead, it only means that the tool had difficulty with identifying that site. Instead of intersecting peaks, let's use a more quantitative approach for differential analysis using [MAnorm](https://github.com/shao-lab/MAnorm).

MAnorm makes a master list of peaks (merging peaks called from either sample), and compares the actual signal at these peaks. In doing so, it also normalizes to the signal across the total peaks to try to normalize for immunoprecipitation efficiency.

In [ ]:
#We specify both peak files (--p1 and --p2), the format of which is narrowpeak (--pf). We also specify the reads in bam format (--r1 and --r2, and --rf).
!manorm --p1 $wd/Submodule2/Peaks/H3K27ac_ChIPseq_noaux_peaks.narrowPeak --p2 $wd/Submodule2/Peaks/H3K27ac_ChIPseq_aux_peaks.narrowPeak --pf narrowpeak --r1 $wd/Submodule2/Filtering/H3K27ac_ChIPseq_noauxdedup.bam --r2 $wd/Submodule2/Filtering/H3K27ac_ChIPseq_auxdedup.bam --rf bam --n1 H3K27ac_noauxin --n2 H3K27ac_auxin -o $wd/Submodule2/MANorm_H3K27ac_ChIPseq
#Let's copy these files to have an easier name
!cp $wd/Submodule2/MANorm_H3K27ac_ChIPseq/output_filters/H3K27ac_noauxin_vs_H3K27ac_auxin_M_above_1.0_biased_peaks.bed $wd/Submodule2/MANorm_H3K27ac_ChIPseq/H3K27ac_higher_in_noauxin.bed
!cp $wd/Submodule2/MANorm_H3K27ac_ChIPseq/output_filters/H3K27ac_noauxin_vs_H3K27ac_auxin_M_below_-1.0_biased_peaks.bed $wd/Submodule2/MANorm_H3K27ac_ChIPseq/H3K27ac_higher_in_auxin.bed

<div class="alert-warning" style="font-size:150%; color:black">
⬇️ Now, within the following block, try to type out the command to create a plot of the signal on loci with higher H3K27ac occupancy in the noauxin sample. ⬇️
</div>

<div class="alert alert-block alert-success" style="font-size:120%">
<span style="color:black">Congrats! You have successfully performed some filtering and downstream analysis! Using what you learned above, try adding some cells to this notebook and, for the differential peaks, visualize in IGV, plot the average signal. 
In the next tutorial, will compare the data to ATAC-seq and RNA-seq data. We highly recommend going through the NIH/NIGMS Sandbox on ATAC-seq and RNA-seq which teach how to generate the differential accessible sites and differentially expressed genes. We'll provide those files and focus on the comparison to chromatin occupancy from ChIP-seq, CUT&RUN, and CUT&Tag.</span>